In [88]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.utils import to_categorical
from keras.callbacks import TensorBoard

import pandas as pd
import numpy as np
import os

import common
from importlib import reload
%matplotlib inline


In [76]:
#
# Parameters
#
split = 6000
max_features = 4000
maxlen = 80
batch_size = 32
embedding_dims = 100
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10


In [77]:
#
# Load the dataset
#
df = pd.read_csv('../dataset/redis_dataset.csv')
train_sentences = df['sentences'][:split]
train_categories = df['readability'][:split]

test_sentences = df['sentences'][split:]
test_categories = df['readability'][split:]
# print(train_sentences)

In [78]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

dictionary = tokenizer.word_index

In [79]:
#
# Load Glove embeddings
#
glove_dir = '/Users/nscsekhar/Desktop/nscsekhar/Desktop/Surya/Personal/MIDS/W266/glove/'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

#
# Get Embedding matrix
#
embedding_matrix = np.zeros((max_features, embedding_dims))
for word, i in dictionary.items():
    if i < max_features:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [80]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(train_sequences, maxlen=maxlen)
x_test = sequence.pad_sequences(test_sequences, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (6000, 80)
x_test shape: (3849, 80)


In [83]:
model = Sequential()
model.add(Dense(hidden_dims, input_shape=(maxlen,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 250)               20250     
_________________________________________________________________
activation_19 (Activation)   (None, 250)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 250)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 251       
_________________________________________________________________
activation_20 (Activation)   (None, 1)                 0         
Total params: 20,501
Trainable params: 20,501
Non-trainable params: 0
_________________________________________________________________
None


In [89]:
tbCallback = TensorBoard(log_dir='./logs/rnn', histogram_freq=0,
                          write_graph=True, write_images=False)
metrics = common.Metrics()

model.fit(x_train, (train_categories),
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1, 
                    validation_data=(x_test, (test_categories)),
                    callbacks=[metrics, tbCallback])

Train on 6000 samples, validate on 3849 samples
Epoch 1/10
6000/6000 [==============================] - 0s 61us/step - loss: 9.6956 - acc: 0.3918 - val_loss: 10.5786 - val_acc: 0.3365
Epoch 2/10
6000/6000 [==============================] - 0s 60us/step - loss: 9.6956 - acc: 0.3918 - val_loss: 10.5786 - val_acc: 0.3365
Epoch 3/10
6000/6000 [==============================] - 0s 62us/step - loss: 9.6956 - acc: 0.3918 - val_loss: 10.5786 - val_acc: 0.3365
Epoch 4/10
6000/6000 [==============================] - 0s 60us/step - loss: 9.6956 - acc: 0.3918 - val_loss: 10.5786 - val_acc: 0.3365
Epoch 5/10
6000/6000 [==============================] - 0s 60us/step - loss: 9.6956 - acc: 0.3918 - val_loss: 10.5786 - val_acc: 0.3365
Epoch 6/10
6000/6000 [==============================] - 0s 61us/step - loss: 9.6956 - acc: 0.3918 - val_loss: 10.5786 - val_acc: 0.3365
Epoch 7/10
6000/6000 [==============================] - 0s 58us/step - loss: 9.6956 - acc: 0.3918 - val_loss: 10.5786 - val_acc: 0.3365


In [90]:
reload(common)
metrics.print_all()

Precision: 0.336451026240582
Recall: 1.0
F1: 0.5034992223950233


In [91]:
# Here's how to generate a prediction on individual examples

for i in range(100):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = np.argmax(prediction)
#     print(train_sequences.iloc[i][:50], "...")
#     print(test_sentences.iloc[i])
    print('Actual label: ', test_categories.iloc[i])
    print("Predicted label: ", predicted_label)
    print("")

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  1
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  1
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  1
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  1
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label:  0
Predicted label:  0

Actual label